In [1]:
partition = 200

In [2]:
import sys
from train import main
from itertools import product  
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

n_tree_values = [5, 10, 15, 20]
tree_depth_values = [3, 5, 10]
batch_size_values = [256, 512, 1000]

best_score = 0
best_config = {}

for n_tree, tree_depth, batch_size in product(n_tree_values, tree_depth_values, batch_size_values):
    print(f"\nRunning: n_tree={n_tree}, tree_depth={tree_depth}, batch_size={batch_size}")
    sys.argv = [
        'train.py',
        '-dataset', f'gtd{partition}',
        '-n_class', '30',
        '-gpuid', '0',
        '-n_tree', str(n_tree),
        '-tree_depth', str(tree_depth),
        '-batch_size', str(batch_size),
        '-epochs', '200',
        '-jointly_training'
    ]
    
    _, _, _ = main()

    # Read best score from file (assumes one run per file)
    result_file = f"results/result_gtd{partition}"
    with open(result_file, "r") as f:
        lines = f.readlines()
        for line in lines:
            if "Best Accuracy" in line:
                acc = float(line.split()[2])
                if acc > best_score:
                    best_score = acc
                    best_config = {
                        'n_tree': n_tree,
                        'tree_depth': tree_depth,
                        'batch_size': batch_size
                    }
print("\nBest hyperparameter configuration:")
print(best_config)
print(f"Best accuracy: {best_score}")



Running: n_tree=5, tree_depth=3, batch_size=256
Use gtd200 dataset
Train Epoch: 1 [0/4200 (0%)]	Loss: 3.401848
Train Epoch: 1 [2560/4200 (59%)]	Loss: 3.376346

Test set: Average loss: 3.3684, Accuracy: 178/1800 (0.0989)

Train Epoch: 2 [0/4200 (0%)]	Loss: 3.365493
Train Epoch: 2 [2560/4200 (59%)]	Loss: 3.351841

Test set: Average loss: 3.3465, Accuracy: 189/1800 (0.1050)

Train Epoch: 3 [0/4200 (0%)]	Loss: 3.336901
Train Epoch: 3 [2560/4200 (59%)]	Loss: 3.335443

Test set: Average loss: 3.3287, Accuracy: 199/1800 (0.1106)

Train Epoch: 4 [0/4200 (0%)]	Loss: 3.313787
Train Epoch: 4 [2560/4200 (59%)]	Loss: 3.300830

Test set: Average loss: 3.3133, Accuracy: 208/1800 (0.1156)

Train Epoch: 5 [0/4200 (0%)]	Loss: 3.297852
Train Epoch: 5 [2560/4200 (59%)]	Loss: 3.290858

Test set: Average loss: 3.2993, Accuracy: 229/1800 (0.1272)

Train Epoch: 6 [0/4200 (0%)]	Loss: 3.278973
Train Epoch: 6 [2560/4200 (59%)]	Loss: 3.295154

Test set: Average loss: 3.2854, Accuracy: 239/1800 (0.1328)

Train Ep

In [3]:
sys.argv = [
    'train.py',
    '-dataset', f'gtd{partition}',
    '-n_class', '30',
    '-gpuid', '0',
    '-n_tree', str(best_config['n_tree']),
    '-tree_depth', str(best_config['tree_depth']),
    '-batch_size', str(best_config['batch_size']),
    '-epochs', '200',
    '-jointly_training'
]

preds, targets, labels = main()


Use gtd200 dataset
Train Epoch: 1 [0/4200 (0%)]	Loss: 3.401522
Train Epoch: 1 [2560/4200 (59%)]	Loss: 3.370126

Test set: Average loss: 3.3463, Accuracy: 346/1800 (0.1922)

Train Epoch: 2 [0/4200 (0%)]	Loss: 3.330381
Train Epoch: 2 [2560/4200 (59%)]	Loss: 3.314807

Test set: Average loss: 3.2934, Accuracy: 348/1800 (0.1933)

Train Epoch: 3 [0/4200 (0%)]	Loss: 3.270272
Train Epoch: 3 [2560/4200 (59%)]	Loss: 3.251226

Test set: Average loss: 3.2525, Accuracy: 394/1800 (0.2189)

Train Epoch: 4 [0/4200 (0%)]	Loss: 3.231114
Train Epoch: 4 [2560/4200 (59%)]	Loss: 3.201622

Test set: Average loss: 3.2200, Accuracy: 446/1800 (0.2478)

Train Epoch: 5 [0/4200 (0%)]	Loss: 3.176658
Train Epoch: 5 [2560/4200 (59%)]	Loss: 3.183562

Test set: Average loss: 3.1921, Accuracy: 498/1800 (0.2767)

Train Epoch: 6 [0/4200 (0%)]	Loss: 3.152667
Train Epoch: 6 [2560/4200 (59%)]	Loss: 3.147028

Test set: Average loss: 3.1671, Accuracy: 545/1800 (0.3028)

Train Epoch: 7 [0/4200 (0%)]	Loss: 3.121067
Train Epoch: 

In [4]:
def plot_confusion_matrix(y_true, y_pred, labels, partition):
    cm = confusion_matrix(y_true, y_pred, labels=range(len(labels)))
    cm_normalized = cm.astype('float') / cm.sum(axis=1, keepdims=True)

    plt.figure(figsize=(18, 16))
    sns.heatmap(cm_normalized,
                annot=True,
                fmt=".2f",
                xticklabels=labels,
                yticklabels=labels,
                cmap="viridis",
                square=True,
                linewidths=0.5,
                cbar_kws={"shrink": 0.8})

    plt.title(f"Normalized Confusion Matrix (Partition gtd{partition})", fontsize=18)
    plt.xlabel("Predicted Label", fontsize=14)
    plt.ylabel("True Label", fontsize=14)
    plt.xticks(rotation=90)
    plt.yticks(rotation=0)
    plt.tight_layout()

    save_path = f"results/confusion_matrix_partition_gtd{partition}.png"
    plt.savefig(save_path, dpi=300)
    plt.close()

    print(f"Saved confusion matrix for partition gtd{partition} to {save_path}")



In [5]:
plot_confusion_matrix(targets, preds, labels, partition)

Saved confusion matrix for partition gtd200 to results/confusion_matrix_partition_gtd200.png
